In [ ]:
import json

In [ ]:
with open('hundreds2.json','rb') as fp:
    data = json.load(fp)

In [ ]:
data.keys()

In [ ]:
X = []
teamV = []
for team in data['teams']:
    for season in team['seasons']:

        cap = season['stadiumCapacity']
        pop = season['pop']
        winp = season['won']/(season['won'] + season['lost'] + season['tied'])
        cost = season['expenses']['salary']['amount']/90000 # payrollEndOfSeason?
        profit = (sum([_['amount'] for _ in season['revenues'].values()])-sum([_['amount'] for _ in season['expenses'].values()]))/90000
        playw = season['playoffRoundsWon']
        X.append([team['abbrev'],season['season'],cap,pop,winp,cost,profit,playw])
        teamV.append([team['abbrev'],season['season'],season['won']])


In [ ]:
import pandas as pd
df =pd.DataFrame(X,columns=['team','season','cap','pop','win','cost','profit','playw'])

In [ ]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
pop_clf = sm.RLM(df.profit,sm.add_constant(df['pop'])).fit()
pop_clf.summary()

In [ ]:
plt.scatter(df['pop'],df.profit,s=5,alpha=0.5)
xn = np.linspace(df['pop'].min(),df['pop'].max(),3)
yn = pop_clf.params[0]  + xn * pop_clf.params[1]
plt.plot(xn,yn,c='k',lw=5)
pop_clf.params

In [ ]:
def expected_profit(pop,stadium):
    return (stadium/25000) * (pop*0.031792 + 0.084825)

In [ ]:
ev = expected_profit(df['pop'],df['cap'])
ev_idx = ev.idxmax()
teamV[ev_idx],ev[ev_idx]

In [ ]:
cost_clf = sm.OLS(df.win,sm.add_constant(df['cost'])).fit()
cost_clf.summary()

In [ ]:
plt.scatter(df['cost'],df.win,s=5,alpha=0.5)
xn = np.linspace(df['cost'].min(),df['cost'].max(),3)
yn = cost_clf.params[0]  + xn * cost_clf.params[1]
plt.plot(xn,yn,c='k',lw=5)
cost_clf.params[0],1/cost_clf.params[1]

In [ ]:
def expected_cost(winp):
    return (winp+0.01474769)*2.010595424083435
ev = expected_cost(df.win)
ev_idx = ev.idxmax()
teamV[ev_idx],ev[ev_idx]

In [ ]:
play_clf = sm.OLS(df.playw,sm.add_constant(df['cost'])).fit()
play_clf.summary()

In [ ]:
plt.scatter(df['cost'],df.playw,s=5,alpha=0.5)
xn = np.linspace(df['cost'].min(),df['cost'].max(),3)
yn = play_clf.params[0]  + xn * play_clf.params[1]
plt.plot(xn,yn,c='k',lw=5)
play_clf.params[0],1/play_clf.params[1]

In [ ]:
def expected_costP(playw):
    return (playw+4.5)*0.2310155
ev = expected_costP(df.win)
ev_idx = ev.idxmax()
teamV[ev_idx],ev[ev_idx]

In [ ]:
ev = df['profit']- expected_profit(df['pop'],df['cap']) + (expected_cost(df['win'])-1) + (expected_costP(df['playw'])-1)

In [ ]:
ev_idx = ev.idxmax()
teamV[ev_idx],ev[ev_idx],ev_idx

In [ ]:
ev_idx = ev.idxmin()
teamV[ev_idx],ev[ev_idx],ev_idx

In [ ]:
df.iloc[946]

In [ ]:
plt.style.use('seaborn-white')
_ = plt.hist(ev,50,density=True)
plt.yticks([],[])

In [ ]:
ev.mean(),ev.median()

In [ ]:
filt = np.array(teamV)[:,1] == '2361'
val = np.array(ev[filt])
team = np.array(teamV)[filt]
for i in np.argsort(val):
    print(val[i],team[i])

In [ ]:
plt.hist(expected_profit(df['pop'],df['cap']).unique())

In [ ]:
df['score'] = ev

In [ ]:
df.groupby('team').mean().sort_values('score')